#### Analyzing the probability of an author writing repeatedly in the same conversation
1. sum up the amounts y == 1 (because an author has answered himself)
2. sum chances of an author seeing himself write
3. calculate a measure of how likely it is that an author sees himself repeated as a test for the nn

In [3]:


import pickle5 as pickle

with open("data/vision_graph_data.pkl", 'rb') as f:
    df = pickle.load(f)


In [4]:
author_count_columns = ["current", "conversation_id", "platform", "y"]
author_df = df[author_count_columns]
author_df

,current,conversation_id,platform,y
0,1540384238922616835,1540323979793137664,twitter,1
1,1540384238922616835,1540323979793137664,twitter,0
2,1540384238922616835,1540323979793137664,twitter,0
3,1540384238922616835,1540323979793137664,twitter,0
4,1540384238922616835,1540323979793137664,twitter,0
...,...,...,...,...
3419843,1533627567378518016,1533622269360087046,twitter,0
3419844,1533627567378518016,1533622269360087046,twitter,0
3419845,1533623769449005058,1533622269360087046,twitter,1
3419846,1533623769449005058,1533622269360087046,twitter,0


In [5]:
author_df = author_df.groupby(["platform", "conversation_id", "current"]).sum()
author_df

y
platform conversation_id     current               
delab    1455538619804831744 72709458             1
         1457613499748831233 572575               1
         1463938900544692229 83448782             1
         1463970148050681857 60138487             1
         1464222274265960454 87771230             1
...                                              ..
twitter  1552558905275858946 1552563036443099137  1
                             1552563089635250176  1
                             1552563118403993600  1
                             1552563228034715649  2
                             1552563434188902400  2

[135767 rows x 1 columns]

In [6]:
author_df = author_df.groupby(["platform", "conversation_id"]).mean()
author_df


y
platform conversation_id              
delab    1455538619804831744  1.000000
         1457613499748831233  1.000000
         1463938900544692229  1.000000
         1463970148050681857  1.000000
         1464222274265960454  1.000000
...                                ...
twitter  1552396506501713925  1.000000
         1552551318866120704  1.869565
         1552558203963154434  1.000000
         1552558701432774659  2.555556
         1552558905275858946  1.235294

[4299 rows x 1 columns]

In [7]:
distinct_view_columns = ["current", "conversation_id", "platform"]
distinct_view_df = df[distinct_view_columns]
distinct_view_df

,current,conversation_id,platform
0,1540384238922616835,1540323979793137664,twitter
1,1540384238922616835,1540323979793137664,twitter
2,1540384238922616835,1540323979793137664,twitter
3,1540384238922616835,1540323979793137664,twitter
4,1540384238922616835,1540323979793137664,twitter
...,...,...,...
3419843,1533627567378518016,1533622269360087046,twitter
3419844,1533627567378518016,1533622269360087046,twitter
3419845,1533623769449005058,1533622269360087046,twitter
3419846,1533623769449005058,1533622269360087046,twitter


In [8]:
distinct_views = distinct_view_df.groupby(["current", "conversation_id", "platform"]).size().to_frame('size')
distinct_views = distinct_views.groupby(["platform", "conversation_id"]).mean()
distinct_views


size
platform conversation_id               
delab    1455538619804831744   1.000000
         1457613499748831233  11.000000
         1463938900544692229  13.000000
         1463970148050681857  29.000000
         1464222274265960454   1.000000
...                                 ...
twitter  1552396506501713925   3.000000
         1552551318866120704  11.956522
         1552558203963154434   3.000000
         1552558701432774659   5.000000
         1552558905275858946   9.000000

[4299 rows x 1 columns]

In [9]:
joined_author_stats = author_df.join(distinct_views)
joined_author_stats = joined_author_stats["y"] / joined_author_stats["size"]
joined_author_stats

platform  conversation_id    
delab     1455538619804831744    1.000000
          1457613499748831233    0.090909
          1463938900544692229    0.076923
          1463970148050681857    0.034483
          1464222274265960454    1.000000
                                   ...   
twitter   1552396506501713925    0.333333
          1552551318866120704    0.156364
          1552558203963154434    0.333333
          1552558701432774659    0.511111
          1552558905275858946    0.137255
Length: 4299, dtype: float64

In [10]:
repetition_probability = joined_author_stats.groupby("platform").mean().to_frame("repetition_probs")
repetition_probability

,repetition_probs
platform,
delab,0.219789
reddit,0.229424
twitter,0.096224
